# RAG(검색-증강 생성) Retrieval-Augmented Generation
- LLM이 외부 데이터를 컨텍스트로 활용.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# x = embedding.embed_query('점심 배불리 먹었더니 졸리다..')
# print(len(x), x)

# VectorStore 에 임베딩 후 저장(In memory)
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding
)

In [6]:
vectorstore.similarity_search(
    '초보자가 키우기 좋은 애완동물 추천해줘', k=2
)

[Document(id='4fcb9946-587e-4b4e-a11f-80c71251c158', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='f297b1be-1062-4671-baa5-b96271b5a50e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

## 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [7]:
%pip install -q pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()


print('원본 pdf의 장수', len(docs))

원본 pdf의 장수 23


In [9]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print('분할 후 청크 수', len(split_docs))

분할 후 청크 수 72


In [10]:
# 3. 임베딩, 4. 벡터스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐', k=4
)

[Document(id='fdd26a0c-e073-4448-ac0f-ee1b5ca3ae32', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-11-09T09:40:27+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-11-09T09:40:27+09:00', 'trapped': '', 'modDate': "D:20231109094027+09'00'", 'creationDate': "D:20231109094027+09'00'", 'page': 0}, page_content='2023년 11월호'),
 Document(id='b4549a54-75a5-4ac2-9f84-6d0ad88a2cdb', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-11-09T09:40:27+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-11-09T09:40:27+09:00', 'trapped': '', 'modDate': "D:20231109094027+09'00'", 'creationDate': "D:20231109094027+09'00'", 'page'

## 검색 증강 단계
1. 사용자 질문 (Query)
2. 검색 (Retrieve)
3. LLM 
4. 최종 답변

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('구글 관련 소식 다 가져와')

'구글은 연말까지 차세대 멀티모달 AI ‘제미니’를 출시할 계획이며, 일부 기업을 대상으로 테스트를 진행 중입니다. 제미니는 인터넷과 구글 서비스에서 수집한 데이터를 활용하여 GPT-4보다 강점을 가지며, AI의 전력 소비와 관련된 논의도 계속되고 있습니다. 또한, 오픈AI는 최신 정보가 반영된 ‘GPT-4 터보’를 공개하고, 개발자용 API와 맞춤형 챗GPT 서비스를 제공하기 시작했습니다.'

In [12]:
%pip install tavily-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# test_tavily.py
import os
from dotenv import load_dotenv
from tavily import TavilyClient

load_dotenv()

tavily_api_key = os.getenv("TAVILY_API_KEY")

if not tavily_api_key:
    print("🚨 .env 파일에서 TAVILY_API_KEY를 찾을 수 없습니다.")
else:
    try:
        print("API 키를 사용하여 Tavily 검색 테스트를 시작합니다...")
        client = TavilyClient(api_key=tavily_api_key)
        response = client.search(query="오늘 대한민국 날씨는?")
        print("✅ 테스트 성공! 검색 결과 일부:")
        print(response['results'][0]['content'])
    except Exception as e:
        print(f"❌ 테스트 실패! 에러 메시지: {e}")

API 키를 사용하여 Tavily 검색 테스트를 시작합니다...
✅ 테스트 성공! 검색 결과 일부:
오늘 서울특별시, 대한민국의 날씨 예보 ; 최고/최저. --/21° ; 바람. 3 km/h ; 습도. 95% ; 이슬점. 22° ; 기압. 1011.5 mb.


In [14]:
import os
from dotenv import load_dotenv

# --- 1. 환경 변수 설정 ---
# .env 파일에서 API 키를 불러옵니다.
load_dotenv()

# API 키들이 설정되었는지 확인
if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    raise ValueError("OPENAI_API_KEY와 TAVILY_API_KEY를 .env 파일에 설정해주세요.")

# --- 2. 필요한 모듈 불러오기 ---
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor

# --- 3. 에이전트 기본 구성 요소 설정 ---
# LLM, 검색 도구, 프롬프트를 설정합니다. 이전 코드와 동일합니다.
llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [TavilySearchResults(k=5, name="tavily_search")]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant specializing in finding and summarizing animation episodes. Use the tavily_search tool to find information."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# --- 4. 에이전트 생성 ---
# 에이전트와 실행기를 생성합니다. 이전 코드와 동일합니다.
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# --- 5. 에이전트에게 내릴 구체적인 작업 명령(TASK) 정의 ---
# ⭐️ 이 부분이 가장 중요합니다! ⭐️
# 에이전트가 명확하게 이해하고 최고의 결과를 내도록 요청을 상세하게 작성합니다.
TASK = """
애니메이션 '검정고무신'에서 주인공 '기영이'가 여자아이와 엮이는 모든 에피소드를 찾아줘.

특히 '경주', '다혜' 등 기영이가 좋아했거나 특별한 관계가 있었던 여자아이들이 등장하는 에피소드를 중심으로 찾아주면 좋겠어.

찾은 각 에피소드에 대해 아래 형식으로 정리해줘:

- **시즌 및 회차**: (예: 시즌 3 5화)
- **에피소드 제목**:
- **주요 줄거리 요약**: (어떤 여자아이와 어떤 이야기가 있었는지 간략하게 요약)
"""

# --- 6. 에이전트 실행 ---
# 정의된 작업을 에이전트에게 전달하여 실행시키고 결과를 받습니다.
print("='검정고무신' 기영이 러브라인 에피소드 검색을 시작합니다...")
print("="*60)

response = agent_executor.invoke({"input": TASK})

print("="*60)
print("✅ 검색 및 정리가 완료되었습니다!\n")
print(f"{response['output']}")

C:\Users\user\AppData\Local\Temp\ipykernel_16904\3220438251.py:21: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tools = [TavilySearchResults(k=5, name="tavily_search")]


='검정고무신' 기영이 러브라인 에피소드 검색을 시작합니다...


> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '검정고무신 기영이 경주 에피소드'}`


[{'title': '검정고무신 등장인물 - 블로그 - NAVER', 'url': 'https://m.blog.naver.com/lbo666/220478990004', 'content': "한 에피소드에서는 기영이가 바나나를 얻어먹기 위해 성철이네 외삼촌 집에 갔다가 바나나가 없어 실망하며 밤늦게 돌아오다 감기에 걸렸는데, 우연히 경주가 바나나를 얻게 되자 그것을 기영이에게 주려고 찾아온 적이 있다.\n\n깍쟁이처럼 보이지만 당찬 면도 있는데, 공포의 쓴맛 편에선 무시무시한 당수 실력으로 반을 휘어잡은 공옥순에게 괴롭힘 당하는 기영이를 보호해주려 하기도 한다.\n\n이 장면에서 기영이가 속마음으로 '경주밖에 없구나 경주야.'라고 했을 정도였으니 사실 플래그는 초반에 꽂아둔 셈.\n\n게다가 한 에피소드에서는 기영이네 가족에게 신붓감으로 공식으로 인정받았을 정도.  \n  \n  \n  \n  \n  \n김다혜\n\n어찌 시즌이 바뀔때마다 머리색깔이 변한다.. 1~2기 에선 보라색, 3기에선 갈색, 4기에선 검은색.\n\n경주와는 반대로 집이 가난한데, 마음씨가 곱다. 기영이를 좋아하며 기영이도 다혜 에게 더 잘대해 준다. [...] 애니판 1, 2기에서는 경주에 비해 별 비중은 없었으나, 3기부터 제대로 나오기 시작한다.\n\n3기 1화에서는 기영이와 다혜의 첫만남을 다룬 에피소드가 있는데, 등교하는 기영이가 개구리를 잡느라고 들고 있던 육성회비 봉투를 내려놓고는 깜빡 잊고 그냥 가자 그 봉투를 주워서 가져간다.\n\n집안 사정이 좋지 않아 부모님께 육성회비를 달라고 하기 어려워 주운 돈으로 육성회비를 내려 했던 것.\n\n그러나 양심 때문에 그 돈을 그냥 갖고만 있었고, 기영이가 그것 때문에 무단결석했다는 